# Voting machine errors during March 2020 primary

### Load python tools

In [266]:
import us
import json
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from pandas.io.json import json_normalize
from altair_saver import save
from altair import datum
import altair as alt
import altair_latimes as lat
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re          
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 34000
pd.set_option('display.max_colwidth', -1)

### Read tickets data

In [267]:
df = pd.read_excel('input/All Tickets for the March Primary Voting Period V2.xlsx').fillna('').dropna(how='all', axis='columns')

### Clean up the dataframe

In [268]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')\
    .str.replace('(', '').str.replace(')', '').str.replace('/','_')

In [269]:
df[['incident_id','priority','vcid','primary_config_asset_tag','comments','incident_childid','location', 'bam_threshold', 'bmd_threshold', 'epollbook_threshold']] \
= df[['incident_id','priority','vcid','primary_config_asset_tag','comments','incident_childid','location', 'bam_threshold', 'bmd_threshold', 'epollbook_threshold']].fillna('').astype(str)

In [270]:
for s in df[['vcid', 'primary_config_asset_tag', 'bam_threshold', 'bmd_threshold', 'epollbook_threshold']]:
    df[f"{s}"] = df[f"{s}"].str.replace('.0', '', regex=False)

In [271]:
df['created_date'] = pd.to_datetime(df['created_date_time']).dt.date
df['created_time'] = pd.to_datetime(df['created_date_time']).dt.time

In [272]:
drop = ['category_description',
       'category_description_de-de', 'category_description_en-us',
       'category_description_es-es', 'category_description_fr-fr',
       'category_description_pt-br', 'incident_category',
       'incident_category_de-de', 'incident_category_en-us',
       'incident_category_es-es', 'incident_category_fr-fr',
       'incident_category_pt-br']

In [273]:
df = df.drop(columns=drop)

### Dates/Times

In [274]:
df_clean = pd.DataFrame(df[df['bmd_related_y_n'] == 'Y'])

In [275]:
df_clean.description = df_clean.description.str.wrap(50)

In [276]:
df_clean.iloc[0]

incident_id                      105529                                                                                                                                                                                                                                                                                                
priority                         4                                                                                                                                                                                                                                                                                                     
status                           Resolved                                                                                                                                                                                                                                                                                              
vcid            